# BI 10


In [ ]:
from prelude import *

## Original


In [ ]:
""" labels & edges """

from dataclasses import dataclass


@dataclass
class BI_10_Record:
    labels: list[str]
    edges: list[tuple[int, int]]


bi_10_short_edges = [(0, 1), (1, 6), (6, 2), (2, 3), (2, 4), (4, 5)] + [
    (6, 7),
    (7, 8),
    (8, 9),
]
bi_10_long_edges = bi_10_short_edges + [(9, 10)]
bi_10_long_labels_template = [
    "country",
    "city",
    "post",  # could use "comment"
    "tag",
    "tag",
    "tagclass",
] + ["person"] * 5
bi_10_short_labels_template = [
    "country",
    "city",
    "post",  # could use "comment"
    "tag",
    "tag",
    "tagclass",
] + ["person"] * 4


def bi_10_wrapper(
    personId: str, country: str, tagclass: str, msgType: str = "post"
) -> list[str]:
    return [
        BaseLabel.PersonId + personId,
        BaseLabel.Country + country,
        BaseLabel.TagClass + tagclass,
        msgType,
    ]


bi_10_task_names = [
    "98214,Tunisia,NascarDriver,4,post",
    "98214,Tunisia,NascarDriver,4,comment",
    "98214,Tunisia,NascarDriver,3,post",
    "98214,Tunisia,NascarDriver,3,comment",
    "4886,Cuba,Thing,4,post",
    "4886,Cuba,Thing,4,comment",
    "4886,Cuba,Thing,3,post",
    "4886,Cuba,Thing,3,comment",
    "60769,France,Politician,4,post",
    "60769,France,Politician,4,comment",
    "60769,France,Politician,3,post",
    "60769,France,Politician,3,comment",
]

bi_10_kwargs = {}
for task_name in bi_10_task_names:
    personId, country, tagclass, length, msgType = task_name.split(",")
    bi_10_kwargs[task_name] = BI_10_Record(
        replace(
            (
                bi_10_long_labels_template
                if length == "4"
                else bi_10_short_labels_template
            ),
            [10, 0, 5, 2] if length == "4" else [9, 0, 5, 2],
            bi_10_wrapper(personId, country, tagclass, msgType),
        ),
        bi_10_long_edges if length == "4" else bi_10_short_edges,
    )

In [ ]:
""" Init Original Query Graph """

for qg_name, meta_record in bi_10_kwargs.items():
    qg_path = f"{BI_10_ORIGINAL_Q_PRE}/{qg_name}.txt"
    labels = meta_record.labels
    edges = meta_record.edges
    qg_emitter(qg_path, labels, edges)

In [ ]:
""" args """

from args import *

bi_10_args_list = [
    original_args_starting + [f"{BI_10_ORIGINAL_Q_PRE}/{task_name}.txt"]
    for task_name in bi_10_task_names
]

In [ ]:
""" exec """

bi_10_time_table = []

bi_10_result_path_list = [
    f"{BI_10_ORIGINAL_L_PRE}/{task_name}.txt" for task_name in bi_10_task_names
]

run_multiple_veq_m_100k(
    bi_10_result_path_list,
    bi_10_task_names,
    bi_10_args_list,
    bi_10_time_table,
)

## Optimized


In [ ]:
""" labels & edges """

bi_10_short_edges_optimized = [(0, 5), (5, 1), (1, 2), (1, 3), (3, 4)] + [
    (5, 6),
    (6, 7),
    (7, 8),
]
bi_10_long_edges_optimized = bi_10_short_edges_optimized + [(8, 9)]

bi_10_task_names_optimized = bi_10_task_names

bi_10_kwargs_optimized = {}
for task_name in bi_10_task_names_optimized:
    personId, country, tagclass, length, msgType = task_name.split(",")
    bi_10_kwargs_optimized[task_name] = BI_10_Record(
        replace(
            (
                bi_10_long_labels_template
                if length == "4"
                else bi_10_short_labels_template
            ),
            [9, 0, 1, 4] if length == "4" else [8, 0, 1, 4],
            bi_10_wrapper(personId, country, tagclass, msgType),
        ),
        bi_10_long_edges_optimized if length == "4" else bi_10_short_edges_optimized,
    )

In [ ]:
""" Init Optimized Query Graph """

for qg_name, meta_record in bi_10_kwargs.items():
    qg_path = f"{BI_10_OPTIMIZED_Q_PRE}/{qg_name}.txt"
    labels = meta_record.labels
    edges = meta_record.edges
    qg_emitter(qg_path, labels, edges)

In [ ]:
""" args """

from args import *

bi_10_args_list_optimized = [
    optimized_args_starting + [f"{BI_10_OPTIMIZED_Q_PRE}/{task_name}.txt"]
    for task_name in bi_10_task_names_optimized
]

In [ ]:
""" exec """

bi_10_time_table_optimized = []

bi_10_result_path_list_optimized = [
    f"{BI_10_OPTIMIZED_L_PRE}/{task_name}.txt"
    for task_name in bi_10_task_names_optimized
]

run_multiple_veq_m_100k(
    bi_10_result_path_list_optimized,
    bi_10_task_names_optimized,
    bi_10_args_list_optimized,
    bi_10_time_table_optimized,
)

In [ ]:
""" Show BI-10 `comparison data-frame` """

print("Comparison between: `original_match` & `optimized_match`")

df = pl.DataFrame(
    {
        "task": bi_10_task_names,
        "original (ms)": bi_10_time_table,
        "optimized (ms)": bi_10_time_table_optimized,
    }
)
df